In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.vehicle'):
    shutil.rmtree('./file/3.vehicle')

os.makedirs('./file/3.vehicle')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                payload = {'partData': {'listing_data_essential': {'parttype': input_.loc[a, 'Type Code'],
                                                                   'partkey': input_.loc[a, 'Part Code']},
                                        'listing_data_supplemental': {'partnumber': input_.loc[a, 'Part Number'],
                                                                      'catalogname': input_.loc[a, 'Manufacturer']}}}
                
                data = {'func': 'getbuyersguide',
                        'payload': json.dumps(payload),
                        'api_json_request': '1'}
    
                
                b = 0
                while True:
                    b += 1
                    try:
                        check_url = input_.loc[a, 'Url']
                        
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))

                        if resp.status_code == 200:
                            dict_ = resp.json()

                            if 'redirect_to_url' not in dict_ and dict_['buyersguidepieces']['title'].startswith('Buyer\'s Guide :') and 'collected_javascript' in dict_:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(dict_['buyersguidepieces']['body'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                if html.xpath('//p/text()') and html.xpath('//p/text()')[0] == 'No applications found.':
                    pass
                else:
                    list_tr = html.xpath('//tr')

                    # = = = = = = = = = = = = = = =

                    if not list_tr:
                        raise
                        
                    # = = = = = = = = = = = = = = =

                    df_temp = pd.DataFrame()
                    for i, tr in enumerate(list_tr):
                        list_text = [text.strip() for text in tr.xpath('./td/text()')]
                    
                        if len(list_text) == 3:
                            if '-' in list_text[2]:
                                [year_begin, year_end] = list_text[2].split('-')
                                for year in range(int(year_begin), int(year_end) + 1):
                                    df_temp_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                                                  'Part Number': input_.loc[a, 'Part Number'],
                                                                  'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC'],
                                                                  'Vehicle No': i + 1,
                                                                  'Make': list_text[0],
                                                                  'Model': list_text[1],
                                                                  'Year': year}])
                                    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                            else:
                                df_temp_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                                              'Part Number': input_.loc[a, 'Part Number'],
                                                              'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC'],
                                                              'Vehicle No': i + 1,
                                                              'Make': list_text[0],
                                                              'Model': list_text[1],
                                                              'Year': int(list_text[2])}])
                                df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                        elif len(list_text) == 2:
                            if '-' in list_text[1]:
                                [year_begin, year_end] = list_text[1].split('-')
                                for year in range(int(year_begin), int(year_end) + 1):
                                    df_temp_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                                                  'Part Number': input_.loc[a, 'Part Number'],
                                                                  'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC'],
                                                                  'Vehicle No': i + 1,
                                                                  'Make': list_text[0],
                                                                  'Model': '',
                                                                  'Year': year}])
                                    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                            else:
                                df_temp_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                                              'Part Number': input_.loc[a, 'Part Number'],
                                                              'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC'],
                                                              'Vehicle No': i + 1,
                                                              'Make': list_text[0],
                                                              'Model': '',
                                                              'Year': int(list_text[1])}])
                                df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                        else:
                            raise
                    
                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Check_Url'] = check_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{check_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Vehicle No', 'Year'],
                                                    ascending=[True, True, False],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.vehicle/{file}',
              f'''./file/3.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://www.rockauto.com/en/parts/melling,30420S,timing+chain+&+component+kit,5756
[剩余数量：5] - [当前时间：19:04:18]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://www.rockauto.com/en/parts/melling,30250S,timing+chain+&+component+kit,5756
[剩余数量：4] - [当前时间：19:04:18]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://www.rockauto.com/en/parts/melling,30460S,timing+chain+&+component+kit,5756
[剩余数量：3] - [当前时间：19:04:18]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://www.rockauto.com/en/parts/melling,30250SX,timing+chain+&+component+kit,5756
[剩余数量：2] - [当前时间：19:04:18]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.https://www.rockauto.com/en/parts/melling,30460SB,timing+chain+&+component+kit,5756
[剩余数量：1] - [当前时间：19:04:19]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.https://www.rockauto.com/en/parts/melling,30460SA,timing+chain+&+component+kit,5756
[剩余数量：0] - [当前时间：19:04:19]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 8.https://www.rockauto.com/en/parts/melling,3

Progress: 100%|███████████████████████████████| 2/2 [00:00<00:00, 418.41it/s]

Done ~
